# Dask and kugupu

The generation of the coupling matrix is the most time consuming calculation done in kugupu.
Luckily as there is no dependency in calculating different frames, the problem is embarrassingly parallel.
Kugupu is able to use the `dask.distributed` package to calculate many frames in parallel.

In [6]:
import kugupu as kgp
import MDAnalysis as mda
from dask import distributed

In [3]:
u = mda.Universe('./datafiles/C6.data', './datafiles/C6.dcd')

u.add_TopologyAttr('names')
namedict = {
    1.008: 'H',
    12.011: 'C',
    14.007: 'N',
    15.999: 'O',
    32.06: 'S',
}
for m, n in namedict.items():
    u.atoms[u.atoms.masses == m].names = n

All that is required is a `distributed.Client` to assign the work to.
Here we create a Client running on our local machine, however it is also possible to use a much more powerful Client.

In [4]:
c = distributed.Client()
c

Client Scheduler: tcp://127.0.0.1:52349 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


The generation of results uses the same function,
but we include the `client=` keyword to make the calculation happen in parallel.

In [7]:
res = kgp.coupling_matrix(u, 5.0, 'lumo', degeneracy=1, client=c)

2019-08-09T14:30:26.427807+0100 INFO Processing 5 frames
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

In [ ]:
The results generated this way are id